In [1]:
import numpy as np
import numba
from numba import jit , njit

In [2]:
# Global Config Variables
n0 = 1000  # number of p=0 points in metric space
V = n0  # Threshold for p=0
K = 10# No of clusters
A= 5# No of attributes
iterations = 80  # maximum iteration in clustering
runs =120

In [3]:
import numpy as np
import os
import sys
import pandas
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from math import log2
from sklearn import preprocessing
import sys
import timeit


In [4]:
import datetime
import json
import random
from collections import defaultdict
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import cm
#from pyclustering.cluster.kmedians import kmedians
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import os



In [5]:
from scipy.stats import norm
from sklearn.cluster import KMeans
!pip install scikit-learn-extra
from sklearn.metrics import pairwise_distances
from itertools import permutations



  Obtaining dependency information for scikit-learn-extra from https://files.pythonhosted.org/packages/22/2f/86f58cb2bd3f81bdb28555470761c27c0d62a566329aac70e8507cb744fd/scikit_learn_extra-0.3.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/340.5 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/340.5 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------  337.9/340.5 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 340.5/340.5 kB 4.2 MB/s eta 0:00:00


In [16]:
import os

# Specify the directory
directory_path = r"C:\Users\fedev\Downloads\ParTree-main\ParTree-main\Competitors\algorithmic fairness\Datasets"

# List all files in the directory
files_in_directory = os.listdir(directory_path)
print("Files in directory:")
for file in files_in_directory:
    print(file)

# Check if 'adult_p.csv' is in the list
if 'adult_p.csv' in files_in_directory:
    print("File 'adult_p.csv' is visible in the directory.")
else:
    print("File 'adult_p.csv' is NOT visible in the directory.")


Files in directory:
adult_p.csv
bank-full_p_6col.csv
bank-full_p_nodiv_6col.csv
data_preprocess.ipynb
diabetic_data_p.csv
File 'adult_p.csv' is visible in the directory.


In [17]:
print(os.getcwd())
def load_Adult(data_dir=''):

    data_dir = data_dir
    _path = 'adult_p.csv'
    data_path = os.path.join(data_dir, _path)

    K = 10

    df = pandas.read_csv(data_path, sep=',')
    #print(df.head())
    #print(len(df))
    
    return df
load_Adult("C:/Users/fedev/Downloads/ParTree-main/ParTree-main/Competitors/algorithmic fairness/Datasets")


C:\Users\fedev\Downloads\ParTree-main\ParTree-main\Competitors\algorithmic fairness\Ablation Study\Impact of center order


,age,fnlwgt,education_num,capital_gain,hours_per_week,type
0,0.019622,-0.022667,-0.680470,0.725976,0.094976,0
1,0.296610,-0.787368,-0.357412,0.402068,-0.051703,0
2,-0.083487,-0.069366,0.479827,-0.822414,-0.285691,0
3,0.636832,-0.021345,0.256530,-0.721425,-0.087912,0
4,-0.393080,-0.017952,0.713519,0.574969,-0.073938,1
...,...,...,...,...,...,...
32556,-0.642387,-0.149353,0.484008,0.564430,-0.110399,1
32557,0.183154,-0.062405,-0.590824,-0.739884,-0.257021,0
32558,0.927796,-0.023090,-0.233823,-0.273761,-0.095099,1
32559,-0.577725,-0.786633,0.052733,-0.199630,-0.069348,0


In [19]:
df=load_Adult("C:/Users/fedev/Downloads/ParTree-main/ParTree-main/Competitors/algorithmic fairness/Datasets")
df= df.round(decimals=5)
print(len(df))
df = df.dropna()
print(len(df))
#df['type'] = df['type']-1
typ = df['gender'].values
#print(len(typ))
#print(df.head(10))
c1 = np.count_nonzero(typ == 0)
c2 = np.count_nonzero(typ == 1)

print(c1/(c1+c2))
print(c2/(c1+c2))

print(c1)
print(c2)
dfDropped = df.drop(columns=['gender'])


32561
32561


KeyError: 'gender'

In [20]:
def dual_print(f,*args,**kwargs):
    #print(*args,**kwargs)
    print(*args,**kwargs,file=f)

def load_dataset(csv_name):
    # read the dataset from csv_name and return as pandas dataframe
    df = pd.read_csv(csv_name, header=None)
    return df


def k_random_index(df,K):
    # return k random indexes in range of dataframe
    return random.sample(range(0, len(df)), K)


def find_k_initial_centroid(df,K):
    centroids = []    # make of form [ [x1,y1]....]

    rnd_idx = k_random_index(df,K)
    #print(rnd_idx)
    for i in rnd_idx:
        coordinates =[]
        for a in range(0,A):
            coordinates.append(df.loc[i][a])
        centroids.append(coordinates)   #df is X,Y,....., Type

    return centroids

#nOt using
def calc_distance(x1, y1, x2, y2):
    # returns the euclidean distance between two points
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

def calc_distance_a(centroid, point):
    #print('çalculating distance\n')

    sum_ = 0

    for i in range(0, len(centroid)):
        sum_ = sum_ + (centroid[i]-point[i])**2

    return sum_ #**0.5

@njit(parallel=False)
def find_distances_fast(k_centroids, df):
    dist = np.zeros((len(k_centroids),len(df),A+2),np.float64)
    Kcnt = 0 
    for c in k_centroids:  #K-centroid is of form [ c1=[x1,y1.....z1], c2=[x2,y2....z2].....]
        l = np.zeros((len(df),A+2),np.float64)
        
        index = 0 
        for row in df:                # row is now x,y,z......type
            # append all coordinates to point
            dis = np.sum((c- row[:A])**2)#calc_distance_a(c, point)
            #Processing the vector for list
            row_list = np.array([dis])
            #append distance or l norm
            row_list = np.append(row_list,row[:A+1])
            #append all coordinates #append type of this row
  
            l[index] = row_list
            index = index + 1
            #[dist, X, Y,....Z , type]
            # l contains list of type [dist,X,Y.....,Z,type] for each points in metric space
        dist[Kcnt]= l
        Kcnt = Kcnt + 1

    # return dist which contains distances of all points from every centroid

    return dist

def find_distances(k_centroids, df):
    dist = []
    for c in k_centroids:  #K-centroid is of form [ c1=[x1,y1.....z1], c2=[x2,y2....z2].....]
        l = []
       # for row in df:

        for index, row in df.iterrows():                # row is now x,y,z......type
            point =[]
            for a in range(0, A):
                point.append(row.iloc[a])  # append all coordinates

            dis = calc_distance_a(c, point)
            #Processing the vector for list
            row_list = [dis]
            #append distance or l norm

            for a in range(0, A):
                row_list.append(row.iloc[a])   #append all coordinates
            #print(row.iloc[a+1])
            row_list.append(row.iloc[a+1])   #append type of this row

            l.append(row_list)
            #l.append([calc_distance(c[0], c[1], row[0], row[1]), row[0], row[1], row[2]])  # [dist, X, Y,....Z , type]
            # l contains list of type [dist,X,Y.....,Z,type] for each points in metric space
        dist.append(l)

    # return dist which contains distances of all points from every centroid

    return dist


def sort_and_valuation(dist):
    sorted_val = []
    for each_centroid_list in dist:
        each_centroid_list_sorted = sorted(each_centroid_list, key=lambda x: (x[A+1], x[0]))  # A+1 is index of type , 0 is dist
        sorted_val.append(each_centroid_list_sorted)

        # sort on basis of type & then dist.
        # Now all whites are towards start and all black are after white as they have additional V added to their valuation
        # Among the whites, the most closest is at start of list as it has more valuation.
        # Similarly sort the black points among them based on distance as did with white

    return sorted_val


def clustering(sorted_valuation, hashmap_points,K):
    n = len(hashmap_points.keys())  # total number of points in metric space
     
    cluster_assign = []

    for i in range(0, K):
        cluster_assign.append([])  # initially all clusters are empty
    map_index_cluster = []
    for i in range(0,K+2):
        map_index_cluster.append(0)
        #initially check all sorted evaluation from 0th index 
    number_of_point_alloc = 0
    curr_cluster = 0
    # until all points are allocated
    while number_of_point_alloc != n:  # As convergence is guaranteed that all points will be allocated to some cluster set
        start_inde = map_index_cluster[curr_cluster % K]
        
        for inde in range(start_inde,len(sorted_valuation[curr_cluster % K])):
            each = sorted_valuation[curr_cluster % K][inde]
            # each is (dist,X,Y,....Z,type)
            if hashmap_points[tuple(each[1: -1])] == 0:    # each is (dist, X,Y,....Z, type)
                cluster_assign[curr_cluster].append(each)
                hashmap_points[tuple(each[1: -1])] = 1
                number_of_point_alloc += 1
                map_index_cluster[curr_cluster % K] = inde  #next time start from here as isse prev all allocated
                break

        curr_cluster = (curr_cluster + 1) % K

    return cluster_assign


def update_centroids_median(cluster_assign,K):
    new_centroids = []
    for k in range(0, K):
        cAk =  np.array(cluster_assign[k])
        cAk = np.delete(cAk,[0,-1],axis=1)
        if len(cAk) %2 ==0 and len(cAk)>0: 
            cc = [np.median(np.array(cAk[:-1])[:,cl]) for cl in range(0,cAk.shape[1])]
            new_centroids.append(cc)
        elif len(cAk) %2 !=0 and len(cAk)>0:
            cc = [np.median(np.array(cAk)[:,cl]) for cl in range(0,cAk.shape[1])]
            new_centroids.append(cc)
        elif len(cAk)==0:
            print("Error: No centroid found updation error")
    return new_centroids
        


def update_centroids(cluster_assign,K):

    new_centroids = []
    for k in range(0, K):

        sum_a = []

        for i in range(0, A):
            sum_a.append(0)

        for each in cluster_assign[k]:
            sum_a = [sum(x) for x in zip(sum_a, each[1:-1])]
            #each is (dist,X,Y,.....Z,type)
        new_coordinates = []
        for a in range(0, A):
            new_coordinates.append(sum_a[a] / len(cluster_assign[k]))
        new_centroids.append(new_coordinates)
        k=k+1



    return new_centroids


def calc_clustering_objective(k_centroid, cluster_assign,K):
    cost = 0

    for k in range(0, K):

        for each in cluster_assign[k]:  #each is (dist, X,Y,....,Z,type)
            dd = calc_distance_a(k_centroid[k], each[1:-1])
            cost = cost + (dd)

    return cost


def calc_fairness_error(df, cluster_assign,K):
    U = []  # distribution of each type in original target dataset for each J = 0 , 1....
    P_k_sum_over_j = []  # distribution in kth cluster  sum_k( sum_j(   Uj * j wale/total_in_cluster ) )

    f_error = 0
    cnt_j_0 = 0
    cnt_j_1 = 0
  #  cnt_j_2 = 0
    cnt = 0
    for index, row in df.iterrows():
        if row.iloc[-1] == 1:
            cnt_j_1 += 1
        elif row.iloc[-1] == 0:
            cnt_j_0 += 1
      #  elif row.iloc[-1] == 2:
        #    cnt_j_2 += 1
            
        cnt += 1

    U.append(cnt_j_0 / cnt)
    U.append(cnt_j_1 / cnt)
    #U.append(cnt_j_2 / cnt)

   

    for k in range(0, K):  # for each cluster

        for j in range(0, len(U)):   #for each demographic group

            cnt_j_cluster = 0
            cnt_total = 0

            for each in cluster_assign[k]:
                if int(each[-1]) == j:    #each is (dist,X, Y.....,Z,type)
                    cnt_j_cluster += 1
                cnt_total += 1
                
            if cnt_j_cluster !=0 and cnt_total != 0:
                P_k_sum_over_j.append(-U[j] * np.log((cnt_j_cluster / cnt_total)/U[j]))
            else:
                P_k_sum_over_j.append(0)  #log(0)=0 considered

    for each in P_k_sum_over_j:
        f_error += each

    return f_error


def calc_balance(cluster_assign,K):
    S_k = []  # balance of each k cluster
    balance = 0  # min (S_k)

    for k in range(0, K):
        cnt_j_0 = 0
        cnt_j_1 = 0
       # cnt_j_2 = 0
        cnt = 0
        for each in cluster_assign[k]:

            if int(each[-1]) == 1:
                cnt_j_1 += 1
            elif int(each[-1]) == 0:
                cnt_j_0 += 1
           # elif int(each[-1]) == 2:
           #     cnt_j_2 += 1
                
            cnt += 1

        if cnt_j_0 != 0 and cnt_j_1 != 0 :#and cnt_j_2!= 0:
            S_k.append(min([cnt_j_0 / cnt_j_1, cnt_j_1 / cnt_j_0 ]))#, cnt_j_1 / cnt_j_2 , cnt_j_2 / cnt_j_1 , cnt_j_0 / cnt_j_2, cnt_j_2 / cnt_j_0 ]))
        elif cnt_j_0 == 0 or cnt_j_1 ==0  :#or cnt_j_2==0:
            S_k.append(0)



    balance = min(S_k)

    return balance


In [23]:


def main():
    # Step1 : Load the dataset
            
    list_fair_K=[]
    list_obj_K =[]       
    list_balance_K=[]
    
    os.makedirs('Adult_kmeans_permu')
    
    for kk in [10]:#2,5,10,15,20,30,40]:
        K = kk
        
        print(" K=="+str(K)+"  ")
        
        list_fair_run=[]
        list_obj_run =[]       
        list_balance_run=[]
        seeds = [0,100,200,300,400,500,600,700,800,900,1000,1100]
      
        for run in range(0,runs):
            np.random.seed(seeds[run])
            random.seed(seeds[run])
            f = open('Adult_kmeans_permu/K_'+str(K)+'_run_'+str(run)+'_output.txt', 'a')
            print("+"*100)
            print('                        RUN  : '+ str(run))


            list_fair_iter=[]
            list_obj_iter =[]
            list_balance_iter=[]

            # Step2 : Find initial K random centroids using k_random_index(df) & find_k_initial_centroid(df)
            k_centroid= find_k_initial_centroid(df,kk)
            k_centroid_permu = list(permutations(k_centroid))
            random.shuffle(k_centroid_permu)
            k_centroid = k_centroid_permu[0]
            permu_index = 0
            max_permu_len =100 #len(k_centroid_permu)
            print("Number of Permutations : "+str(max_permu_len))
            
            prev_assignment =[]
            cluster_assignment = []

            for i in range(0, K):
                cluster_assignment.append([])  # initially all clusters are empty

            sum_time = 0
            curr_itr = 0
            prev_objective_cost=-1
            objective_cost = 0
                # Step3 : Find distances from the centroids using find_distances() with list of [ [x1,y1,z1..] , [x2,y2,z2..]....] centroids format list
            while True:# and prev_objective_cost != objective_cost:

                start = time.process_time()#timeit.default_timer()
          
                dual_print(f,'Calulating distance for iteration : '+ str(curr_itr)+'\n')
                df1 = df.values
                k_centroids1= np.array(k_centroid)

                dist = find_distances_fast(k_centroids1, df1)
                dual_print(f,'Finished calc distance for iteration : '+ str(curr_itr)+'\n')
                # Step4 :  Find Valuation matrix for all centroids using sort_and_valuation()
                dual_print(f,'Calulating Valuation for iteration : '+ str(curr_itr)+'\n')

                valuation = sort_and_valuation(dist)
                dual_print(f,'Finished Valuation for iteration : '+ str(curr_itr)+'\n')

                #Step5 : Perform clustering using valuation matrix & hashmap of all points in metric
                hash_map = {}
                for index, row in df.iterrows():
                    temp = tuple(row[:-1])
                    hash_map.update({tuple(row[:-1]): 0})   #dict is of form { (x,y): 0 , ....}
                dual_print(f,'Finding clusters for iteration : '+ str(curr_itr)+'\n')
                prev_assignment = cluster_assignment
                cluster_assignment = clustering(valuation, hash_map,K)
                
                print("cluster_assignment", cluster_assignment)

                dual_print(f,'Finished finding cluster for iteration : '+ str(curr_itr)+'\n')

              #  print("Finding balance ")
                balance = calc_balance(cluster_assignment,K)
                f_error = calc_fairness_error(df, cluster_assignment,K)
                clustering_cost = calc_clustering_objective(k_centroid,cluster_assignment,K)
                    
                objective_cost = np.round(clustering_cost,3)


              
                list_balance_iter.append(str(balance))
                list_obj_iter.append(str(objective_cost))
                list_fair_iter.append(str(f_error))

                dual_print(f,'balance : ' + str(balance) + '\n')
                dual_print(f,'Fairness Error : ' + str(f_error) + '\n')
                dual_print(f,'Clustering Objective/Cost ' + str(clustering_cost) + '\n')

                #Step6 : Print the cluster assignments
                #Step7 : Find new centroids using mean of all points in current assignment
                
                stopFlag =0
                if permu_index < max_permu_len-1:
                    permu_index += 1
                    k_centroid = k_centroid_permu[permu_index] #update_centroids(cluster_assignment,K)
                else:
                    stopFlag =1
                
                
                dual_print(f,'Finished centroid updation for iteration : '+ str(curr_itr)+'\n')
                dual_print(f,'Iteration No: '+str(curr_itr)+' : updated centroid are : '+ str(k_centroid))
                #Step8 : Repeat from Step3 until clusters are same or iterations reach upper limit
                stop = time.process_time()#timeit.default_timer()
                sum_time += (stop - start)
                dual_print(f,'Time for iteration : ' + str(curr_itr) + ' is  ' + str(stop - start) + '\n')
                
            
                curr_itr += 1
                
                if stopFlag==1:
                    break
                    
                dual_print(f,'-----------------------------Finished-----------------------------------------------\n')


              
            print('Total time taken to converge '+ str(sum_time)+'\n')
            print('Iterations total taken for convergence : '+str(curr_itr)+'\n')

            dual_print(f,'Total time taken is '+ str(sum_time)+'\n')
            dual_print(f,'Iterations total : '+str(curr_itr-1))
           
            #Step 10 : Find balance , fairness error , and clustering objective or cost

            balance_converged = calc_balance(cluster_assignment,K)
            f_error_converged = calc_fairness_error(df, cluster_assignment,K)
            clustering_cost_converged = calc_clustering_objective(k_centroid,cluster_assignment,K)

            print("\nCost variation over iterations")
            print(list_obj_iter)
            print("\nBalance variation over iterations")
            print(list_balance_iter)
            print("\nFairness error over iterations")
            print(list_fair_iter)
            print('\n')
            print('Final converged balance : ' + str(balance_converged) + '\n')
            print('Final Converged Fairness Error : ' + str(f_error_converged) + '\n')
            print('Final converged Clustering Objective/Cost ' + str(clustering_cost_converged) + '\n')

            dual_print(f,'Converged balance : ' + str(balance_converged) + '\n')
            dual_print(f,'Converged Fairness Error : ' + str(f_error_converged) + '\n')

            dual_print(f,'Converged Clustering Objective/Cost ' + str(clustering_cost_converged) + '\n')
            
            f.close()
            run  = run +1
            list_obj_run.append(clustering_cost_converged)
            list_fair_run.append(f_error_converged)
            list_balance_run.append(balance_converged)
        
        print("@"*70)
        print("Cost variations over run")
        print(str(list_obj_run))
        print("balance variations over run")
        print(str(list_balance_run))
        print("fairness error over run")
        print(str(list_fair_run))
        print("#"*30)
        print("Mean Cost variations over run")
        print(str(np.mean(np.array(list_obj_run))))
        print("Std Dev Cost variations over run")
        print(str(np.std(np.array(list_obj_run))))
        print("#"*30)
        
        list_obj_K.append(np.mean(np.array(list_obj_run)))
        list_fair_K.append(np.mean(np.array(list_fair_run)))
        list_balance_K.append(np.mean(np.array(list_balance_run)))
        
    print("%"*70)
    print("Cost variations over K")
    print(str(list_obj_K))
    print("balance variations over K")
    print(str(list_balance_K))
    print("fairness error over K")
    print(str(list_fair_K))
    print("#"*30)

     

In [24]:

import time
if __name__ == "__main__":
    main()
    

FileExistsError: [WinError 183] Impossibile creare un file, se il file esiste già: 'Adult_kmeans_permu'